1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 백터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
%pip install --upgrade --quiet  docx2txt
%pip install --upgrade --quiet  langchain-community

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

document_list

In [ ]:
%pip install langchain-chroma

In [5]:
from langchain_chroma import Chroma

database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name="tax", persist_directory="./chroma")


In [ ]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가?'
retrieved_docs = database.similarity_search(query, k=3)
retrieved_docs

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini")

# prompt = f"""[Identity]

# - 당신은 최고의 한국 소득세 전문가 입니다.
# - [Context]를 참고해서 사용자의 질문에 답변해주세요.

# [Context]
# {retrieved_docs}

# Question: {query}
# """

# ai_message = llm.invoke(prompt)
# ai_message.content

In [ ]:
%pip install --upgrade --quiet langchain-community langchain langchain-openai faiss-cpu

In [10]:
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain import hub
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma

from dotenv import load_dotenv

load_dotenv()

# 모델 설정
llm = ChatOpenAI(model_name="gpt-4o-mini")

# Knowledge Base 구성을 위한 데이터 생성
# - RecursiveCharacterTextSplitter를 활용한 데이터 chunking
#   - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
#   - 비용 감소와 답변 생성시간 감소의 효과
#   - LangChain에서 다양한 TextSplitter들을 제공
# - chunk_size 는 split 된 chunk의 최대 크기
# - chunk_overlap은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)
loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

# 데이터를 처음 저장할 때 
# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name="tax", persist_directory="./chroma")

# 이미 저장된 데이터를 사용할 때 
database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

# 답변 생성을 위한 Retrieval
query = '연봉 5천만원인 직장인의 소득세는 얼마인가?'

# Augmentation을 위한 Prompt 활용
# - Retrieval된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용
prompt = hub.pull("rlm/rag-prompt")

# 답변 생성
qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

ai_message = qa_chain.invoke({"query": query})
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가?',
 'result': '연봉 5천만원인 직장인의 소득세는 약 700만원에서 800만원 사이로 예상됩니다. 이는 기본 공제와 세율을 적용한 대략적인 계산입니다. 정확한 금액은 개인의 공제 항목에 따라 달라질 수 있습니다.'}

In [22]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o-mini")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)
loader = Docx2txtLoader("./tax.docx")
docs = loader.load_and_split(text_splitter=text_splitter)
embedding = OpenAIEmbeddings(model="text-embedding-3-large")
vetorstore = Chroma.from_documents(documents=docs, embedding=embedding)
retriever = vetorstore.as_retriever()

system_prompt = """
    - 당신은 최고의 한국 소득세 전문가 입니다.
    - 답변은 영어로 제공해주세요.
    - [Context]를 참고해서 사용자의 질문에 답변해주세요.
    
    [Context]
    {context}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

input = '연봉 5천만원인 직장인의 소득세는 얼마인가?'
result = rag_chain.invoke({"input": input})
result

ChatPromptTemplate(input_variables=['context', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\n    - 당신은 최고의 한국 소득세 전문가 입니다.\n    - 답변은 영어로 제공해주세요.\n    - [Context]를 참고해서 사용자의 질문에 답변해주세요.\n    \n    [Context]\n    {context}\n')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [26]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'retrieval-qa-chat', 'lc_hub_commit_hash': 'b60afb6297176b022244feb83066e10ecadcda7b90423654c4a9d45e7a73cebc'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='Answer any use questions based solely on the context below:\n\n<context>\n{context}\n</context>')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])